#**순환 신경망 구현**

이번 순환 신경망 구현에서 사용할 데이터는 '텍스트 데이터'이다.   
텍스트를 순환 신경망에 사용하기 위해서는 적절한 변환이 필요하다.   

###**데이터셋 준비**

IMDB 데이터셋은 인터넷 영화 데이터베이스에서 수집한 영화리뷰 데이터이다.   
순환 신경망으로 긍정적/부정적 리뷰를 판별한다.   
훈련세트 25000개, 테스트세트 25000개로 구성되어 있고, 훈련세트에서 5000개를 떼어 검증세트로 활용할 것이다.

IMDB 데이터셋에는 리뷰에 포함된 80000개 이상의 단어를 고유한 정수로 미리 변환해 두었다.

In [1]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.datasets import imdb

(x_train_all, y_train_all), (x_test, y_test) = imdb.load_data(skip_top=20, num_words=100)

17473536/17464789 [==============================] - 1s 0us/step


- skip_top 파라미터는 가장 많이 등장하는 단어들 중 건너뛸 단어의 개수를 정한다.   
(예를 들어 a, is, the와 같은 분석에 유용하지 않으므로 건너뛴다.)   

- num_words 파라미터는 훈련에 사용할 단어의 개수를 지정한다.   
(이 실습에서는 메모리를 절약하기 위해 100개의 단어만 훈련에 사용한다.)

In [2]:
print(x_train_all.shape, y_train_all.shape)

(25000,) (25000,)


In [7]:
'''
  단어는 정수로 표현이 되는데, 이 정수들은 영단어를 고유한 정수에 일대일 대응한 것으로
  BoW(Bag of Word) 혹은 어휘사전이라고 부른다.
  숫자 '2'는 어휘사전에 없는 단어를 일컫는데, 이는 앞에서 가장 많이 사용되는 단어 20개를 건너뛰고 100개의 단어만 선택했기 때문이다.
  따라서 (0 ~ 1), (101 ~ )의 정수에 대응하는 단어는 모두 '2'로 표시된다.
  숫자 '0', '1'은 각 패딩과 글의 시작을 나타내는데, 이 숫자들을 제외하고 훈련세트를 만든다.
'''
print(x_train_all[0])

[22, 43, 65, 66, 36, 25, 43, 50, 35, 39, 38, 50, 22, 22, 71, 87, 43, 38, 76, 22, 62, 66, 33, 38, 25, 51, 36, 48, 25, 33, 22, 28, 77, 52, 82, 36, 71, 43, 26, 46, 88, 98, 32, 56, 26, 22, 21, 26, 30, 51, 36, 28, 92, 25, 65, 38, 88, 32, 32]


In [5]:
for i in range(len(x_train_all[0])):
  x_train_all[0] = [w for w in x_train_all[0] if w > 2]  # x_train_all[0] 요소 중 2보다 큰 값만 저장

print(x_train_all[0])

[22, 43, 65, 66, 36, 25, 43, 50, 35, 39, 38, 50, 22, 22, 71, 87, 43, 38, 76, 22, 62, 66, 33, 38, 25, 51, 36, 48, 25, 33, 22, 28, 77, 52, 82, 36, 71, 43, 26, 46, 88, 98, 32, 56, 26, 22, 21, 26, 30, 51, 36, 28, 92, 25, 65, 38, 88, 32, 32]


In [6]:
'''
  훈련세트를 이해하기 쉽게 영단어로 바꿔보자.
  정수를 영단어로 바꾸기 위한 "어휘사전"은 get_word_index() 메서드로 내려받을 수 있다.
  이 메서드는 영단어와 정수로 구성된 딕셔너리를 리턴한다.
  다음은 영단어 'movie'에 대응하는 정수 17을 보여준다.
'''
word_to_index = imdb.get_word_index()
word_to_index['movie']

1654784/1641221 [==============================] - 0s 0us/step


17

In [9]:
'''
  훈련세트의 정수는 3 이상부터 영단어를 의미하므로 3을 뺀 값을 어휘사전의 인덱스로 사용해야 한다.
'''

index_to_word = {word_to_index[k]: k for k in word_to_index}

for w in x_train_all[0]:
  print(index_to_word[w - 3], end=' ')

film just story really they you just there an from so there film film were great just so much film would really at so you what they if you at film have been good also they were just are out because them all up are film but are be what they have don't you story so because all all 

In [11]:
'''
  훈련세트의 입력 데이터는 넘파이 배열이 아니라 파이썬 리스트 타입이다.
  각 리뷰의 길이가 달라 샘플의 길이가 다르기 때문이다.

  두 샘플의 길이가 59, 189로 큰 차이를 보인다.
  이러한 경우 모델을 제대로 훈련시킬 수 없다.
'''
print(len(x_train_all[0]), len(x_train_all[1]))

59 189


In [12]:
'''
  훈련세트의 타깃값을 확인해보자.
  이진 분류 문제이므로 1(긍정), 0(부정)으로 표기된다.
'''
print(y_train_all[:10])

[1 0 0 1 0 0 1 0 1 0]


In [13]:
'''
  검증세트를 준비하자.
  25000개의 훈련세트 중 5000개를 분리하여 검증세트로 이용한다.
  넘파이의 permutation() 메서드를 이용해 25000개의 인덱스를 섞은 후
  앞의 20000개는 훈련세트, 나머지는 검증세트로 분리한다.
'''
np.random.seed(42)
random_index = np.random.permutation(25000)

x_train = x_train_all[random_index[:20000]]
y_train = y_train_all[random_index[:20000]]

x_val = x_train_all[random_index[20000:]]
y_val = y_train_all[random_index[20000:]]